In [ ]:
import numpy as np
import random
import math
from itertools import permutations
from google.colab import files
import zipfile
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import shutil
from google.colab import files
import glob
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
# @title
import os
import random
import numpy as np
import torch

def set_global_seed(seed=42):
    # Python's built-in random module
    random.seed(seed)

    # NumPy
    np.random.seed(seed)

    # PyTorch (CPU y GPU)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # Para múltiples GPUs

    # Determinismo en cuDNN (afecta rendimiento)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(True)
    
    # Para reproducibilidad total en transformaciones aleatorias
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'  # Para CUDA >=10.2

set_global_seed(42)


AUTOENCODER

In [ ]:
import numpy as np

lolib = np.load('/kaggle/input/inastancias-mezclado/todo_normalizado_mezclado.npy')
print(lolib.shape)
aleatorio = np.load('/kaggle/input/inastancias-mezclado/instancias_aleatorias.npy')
print(aleatorio.shape)
instancias = np.load('/kaggle/input/inastancias-mezclado/instancias_mezclado.npy')
print(instancias.shape)
train_3, test_3 = train_test_split(instancias, test_size=0.2, random_state=42)
train = train_3
validacion = test_3
test = test_3
set_global_seed(42)
print(train.shape)

In [ ]:
# @title
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

class Autoencoder(nn.Module):
    def __init__(self, input_dim, latent_dim, hidden_dim=80):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, latent_dim),
            nn.Tanh()
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, input_dim),
            nn.Tanh()
        )

    def forward(self, x):
        z = self.encoder(x)
        out = self.decoder(z)
        return out

def ae_loss(recon_x, x):
    return F.mse_loss(recon_x, x, reduction='mean')

def train_autoencoder(model, data, epochs=50, batch_size=64, lr=1e-3):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    for epoch in range(epochs):
        model.train()
        for i in range(0, len(data), batch_size):
            batch = data[i:i+batch_size]
            optimizer.zero_grad()
            recon = model(batch)
            loss = ae_loss(recon, batch)
            loss.backward()
            optimizer.step()

def get_ae_latents(model, data, batch_size=64):
    model.eval()
    latents = []
    with torch.no_grad():
        for i in range(0, len(data), batch_size):
            batch = data[i:i+batch_size]
            z = model.encoder(batch)
            latents.append(z)
    return torch.cat(latents, dim=0)

def train_autoencoder_from_matrices(matrix_data, N, latent_dim=12, hidden_dim=80, epochs=15, batch_size=32, lr=1e-3):
    """
    matrix_data: list or np.ndarray of shape (num_instances, N, N)
    N: size of each square matrix (e.g. 20)
    """
    matrix_array = np.array(matrix_data, dtype=np.float32)

    # Mask to remove diagonal
    mask = ~np.eye(N, dtype=bool)
    masked_data = matrix_array[:, mask]

    # Flatten each masked matrix into a vector
    flat_data = masked_data.reshape(masked_data.shape[0], -1)

    # Convert to torch tensor
    tensor_data = torch.tensor(flat_data, dtype=torch.float32)

    # Define and train the model
    input_dim = tensor_data.shape[1]
    ae_model = Autoencoder(input_dim=input_dim, latent_dim=latent_dim, hidden_dim=hidden_dim)
    train_autoencoder(ae_model, tensor_data, epochs=epochs, batch_size=batch_size, lr=lr)

    # Get latent representation
    latents = get_ae_latents(ae_model, tensor_data, batch_size=batch_size)

    return latents, ae_model

In [ ]:
set_global_seed(42)
z, ae_model = train_autoencoder_from_matrices(train, 20)
print(np.max(z.numpy()))
print(np.mean(z.numpy()))

ALGORITHMS

Profit permutation

In [ ]:
import numpy as np
from numba import njit

@njit
def profit_permutation(A, sigma):
    """
    Compute the profit for a permutation sigma on matrix A,
    summing only the upper triangular part (excluding diagonal).
    """
    total = 0.0
    N = len(sigma)
    for i in range(N):
        for j in range(i + 1, N):  # Upper triangle only
            total += A[sigma[i], sigma[j]]
    return total

2-opt

In [ ]:
@njit
def two_opt_ls(A, rand_indices):
    N = A.shape[0]
    sigma = np.arange(N)
    for i in range(N-1, 0, -1):
        j = rand_indices[N - 1 - i]
        sigma[i], sigma[j] = sigma[j], sigma[i]

    a = profit_permutation(A, sigma)
    improvement = True

    while improvement:
        improvement = False
        for i in range(N):
            for j in range(i+1, N):
                # Reverse segment in-place
                sigma[i:j+1] = sigma[i:j+1][::-1]
                b = profit_permutation(A, sigma)
                if b > a:
                    a = b
                    improvement = True
                    break  # restart
                else:
                    # Undo reversal
                    sigma[i:j+1] = sigma[i:j+1][::-1]
            if improvement:
                break

    return sigma

Swap

In [ ]:
@njit
def swap_ls(A, rand_indices):
    """
    Local search with pairwise swap moves to maximize profit.
    Optimized with Numba and in-place swaps.
    """
    N = A.shape[0]

    # Generate initial random permutation (Fisher-Yates shuffle)
    sigma = np.arange(N)
    for i in range(N - 1, 0, -1):
        j = rand_indices[N - 1 - i]
        sigma[i], sigma[j] = sigma[j], sigma[i]

    current_profit = profit_permutation(A, sigma)
    improvement = True

    while improvement:
        improvement = False
        for i in range(N):
            for j in range(i + 1, N):
                # Swap in-place
                sigma[i], sigma[j] = sigma[j], sigma[i]
                new_profit = profit_permutation(A, sigma)

                if new_profit > current_profit:
                    current_profit = new_profit
                    improvement = True
                    break  # Restart search after improvement
                else:
                    # Undo swap
                    sigma[i], sigma[j] = sigma[j], sigma[i]
            if improvement:
                break

    return sigma

Insert

In [ ]:
@njit
def insert_ls(A, rand_indices):
    N = A.shape[0]
    sigma = np.arange(N)
    for i in range(N-1, 0, -1):
        j = rand_indices[N - 1 - i]
        sigma[i], sigma[j] = sigma[j], sigma[i]

    a = profit_permutation(A, sigma)
    improvement = True

    while improvement:
        improvement = False
        for i in range(N):
            for j in range(N):
                if i == j:
                    continue
                sigma_copy = sigma.copy()
                temp = sigma_copy[i]
                if i < j:
                    for k in range(i, j):
                        sigma_copy[k] = sigma_copy[k+1]
                else:
                    for k in range(i, j, -1):
                        sigma_copy[k] = sigma_copy[k-1]
                sigma_copy[j] = temp

                b = profit_permutation(A, sigma_copy)
                if b > a:
                    sigma = sigma_copy
                    a = b
                    improvement = True
                    break
            if improvement:
                break

    return sigma

Profit

In [ ]:
# @title
def profit(A, algorithm):
    rand_indices = [np.random.randint(0, i+1) for i in range(N-1, 0, -1)]
    sigma = np.array(algorithm(A, rand_indices))
    idx = sigma.astype(int)

    A_sub = A[np.ix_(idx, idx)]
    total_sum = np.sum(np.triu(A_sub, k=1))
    return total_sum

NOVELTY SEARCH

Initialise

In [ ]:
# @title
def initialise(D, N):
    # D[0] = número de instancias, D[1] = valor máximo para la distribución uniforme (-D[1], D[1])
    num_instancias = D[0]
    max_valor = D[1]

    population = np.empty((num_instancias, N, N))

    for idx in range(num_instancias):
        A = np.random.uniform(-max_valor, max_valor, size=(N, N))
        np.fill_diagonal(A, 0)
        population[idx] = A
    return population


Profit portfolio

In [ ]:
# @title
def profit_portfolio(A, portfolio):
    f = np.array([profit(A, algo) for algo in portfolio])
    return f.reshape(-1, 1)

Novelty score

In [ ]:
# @title
import numpy as np
import math

def novelty_score(population, archive, k):
    pop_size = len(population)
    archive_size = len(archive)
    N = population[0].shape[0]

    U = np.zeros((archive_size, 12))
    D = np.zeros((pop_size, 12))

    lower_indices = np.tril_indices(N, k=-1)

    mask = ~np.eye(N, dtype=bool)
    masked_data = archive[:,mask]
    flat = masked_data.reshape(masked_data.shape[0], -1)
    tens = torch.tensor(np.array(flat), dtype=torch.float32)
    ae_model.eval()
    with torch.no_grad():
        U = ae_model.encoder(tens).cpu().numpy().astype(np.float32)

    mask = ~np.eye(N, dtype=bool)
    masked_data = population[:,mask]
    flat = masked_data.reshape(masked_data.shape[0], -1)
    tens = torch.tensor(np.array(flat), dtype=torch.float32)
    ae_model.eval()
    with torch.no_grad():
        D = ae_model.encoder(tens).cpu().numpy().astype(np.float32)

    all_descriptors = np.concatenate((U, D), axis=0)
    total_descriptors = all_descriptors.shape[0]

    scores = np.zeros((pop_size, 1))
    for t in range(pop_size):
        distances = np.linalg.norm(all_descriptors - D[t, :], axis=1)
        self_index = archive_size + t
        distances[self_index] = np.inf
        sorted_dist = np.sort(distances)
        finite_dists = sorted_dist[np.isfinite(sorted_dist)]
        k_eff = min(k, len(finite_dists))
        scores[t] = np.mean(finite_dists[:k_eff])

    return scores

Performance score

In [ ]:
def performance_score(population, portfolio, R=10):
    amount_algorithms = len(portfolio)
    pop_size = population.shape[0]

    performance_scores = np.zeros((pop_size, amount_algorithms))

    for i in range(pop_size):
        # Inicializar acumulador para R ejecuciones
        cumulative_profit = np.zeros(amount_algorithms)

        for r in range(R):
            profit_vec = profit_portfolio(population[i], portfolio)
            profit_vec = profit_vec.flatten()
            cumulative_profit += profit_vec

        # Promediar el resultado sobre R ejecuciones
        performance_scores[i, :] = cumulative_profit / R

    # Ajustar el score del solver en la posición 0
    for i in range(pop_size):
        target_value = performance_scores[i, 0]
        other_values = performance_scores[i, 1:]
        performance_scores[i, 0] = target_value - np.max(other_values)

    return performance_scores

Fitness (novelty* phi + performance* (1-phi))

In [ ]:
def evaluate(population, archive, portfolio, k, phi, R=10):
    pop_size = population.shape[0]

    novelty = novelty_score(population, archive, k)
    performance = performance_score(population, portfolio, R)

    novelty = np.asarray(novelty).flatten()

    target_performance = np.asarray(performance[:, 0]).flatten()

    novelty_std = (novelty - np.mean(novelty)) / np.std(novelty)
    target_performance_std = (target_performance - np.mean(target_performance)) / np.std(target_performance)

    fitness = phi * target_performance_std + (1 - phi) * novelty_std

    return fitness.reshape(-1, 1)

Cross

In [ ]:
# @title
def cross(instance_1, instance_2, N):
    mask = ~np.eye(N, dtype=bool)
    flat_1 = instance_1[mask].flatten()
    flat_2 = instance_2[mask].flatten()

    dimension = N * N - N
    crossover_mask = np.random.randint(0, 2, dimension)

    offspring1 = np.where(crossover_mask, flat_1, flat_2)
    offspring2 = np.where(crossover_mask, flat_2, flat_1)

    return offspring1, offspring2

Mutation

In [ ]:
# @title
def mutation(instance, mutation_rate=0.01):
    instance = instance.copy()
    mask = np.random.rand(len(instance)) < mutation_rate
    instance[mask] = np.random.uniform(-1, 1, np.sum(mask))
    return instance

Array to matrix

In [ ]:
# @title
def array_to_matrix(array, N):
    matrix = np.zeros((N, N))
    idx = 0
    for i in range(N):
        for j in range(N):
            if i != j:
                matrix[i, j] = array[idx]
                idx += 1
    return matrix

Offsprings

In [ ]:
# @title
import numpy as np
import random

# @title
# def offspring(population, archive, portfolio, k, phi, mutation_rate=0.01):
#     pop_size, N, _ = population.shape
#     flat_size = N * N - N

#     fitness_values = evaluate(population, archive, portfolio, k, phi)
#     indexes = np.arange(pop_size)

#     indexes_for_cross = np.zeros(pop_size, dtype=int)
#     for i in range(pop_size):
#         a, b = random.choices(indexes, k=2)
#         indexes_for_cross[i] = a if fitness_values[a] > fitness_values[b] else b

#     parents = random.choices(indexes_for_cross.tolist(), k=2 * pop_size)
#     parents = np.array(parents).reshape(pop_size, 2)

#     offspring_flat = np.zeros((pop_size, flat_size))
#     for i in range(0, pop_size, 2):
#         p1 = population[parents[i, 0]]
#         p2 = population[parents[i, 1]]
#         o1, o2 = cross(p1, p2, N)
#         o1 = mutation(o1)
#         o2 = mutation(o2)
#         offspring_flat[i] = o1
#         offspring_flat[i + 1] = o2

#     offspring_matrix = np.array([array_to_matrix(vec, N) for vec in offspring_flat])
#     return offspring_matrix
    
def offspring(population, archive, portfolio, k, phi, mutation_rate=0.01, R=10):
    pop_size, N, _ = population.shape
    flat_size = N * N - N

    # Calcular fitness para cada individuo
    fitness_values = evaluate(population, archive, portfolio, k, phi, R)
    indexes = np.arange(pop_size)

    # Seleccionar 2 * pop_size padres usando torneo binario
    parents = []
    for _ in range(2 * pop_size):
        a, b = random.choices(indexes, k=2)
        winner = a if fitness_values[a] > fitness_values[b] else b  # Cambia a < si es minimización
        parents.append(winner)
    parents = np.array(parents).reshape(pop_size, 2)

    # Generar descendencia
    offspring_flat = np.zeros((pop_size, flat_size))
    for i in range(0, pop_size, 2):
        p1 = population[parents[i, 0]]
        p2 = population[parents[i, 1]]
        o1, o2 = cross(p1, p2, N)
        o1 = mutation(o1, mutation_rate)
        o2 = mutation(o2, mutation_rate)
        offspring_flat[i] = o1
        offspring_flat[i + 1] = o2

    # Convertir a matrices de adyacencia
    offspring_matrix = np.array([array_to_matrix(vec, N) for vec in offspring_flat])
    return offspring_matrix

Update archive

In [ ]:
# @title
def update_archive(population, archive, portfolio, k, phi, ta, R=10):
    pop_size = len(population)
    pop_novelty = novelty_score(population, archive, k)
    pop_perf = performance_score(population, portfolio, R)

    for i in range(pop_size):
        # 1% de probabilidad de añadir la instancia
        if random.random() < 0.01:
            archive = np.append(archive, [population[i]], axis=0)
        elif pop_perf[i, 0] > 0 and pop_novelty[i] > ta:
            archive = np.append(archive, [population[i]], axis=0)

    return archive

Update solution set

In [ ]:
# @title
def update_ss(population, solution_set, portfolio, phi, tss, R=10):
    pop_size = len(population)
    pop_perf = performance_score(population, portfolio, R)

    for i in range(pop_size):
        if pop_perf[i, 0] > 0:
            novelty = novelty_score(np.array([population[i]]), solution_set, k=1)[0]
            if novelty > tss:
                solution_set = np.append(solution_set, [population[i]], axis=0)

    return solution_set

Novelty search

In [ ]:
# @title
def novelty_search(D, N, k, phi, generations, portfolio, ta, tss, R=10, mutation_rate=0.01, initial_archive = [], population = []):
    if len(population) == 0:
        population = initialise(D, N)
    else:
        population = population

    if len(initial_archive) == 0:
        archive = np.array([random.choice(population)])
    else:
        archive = initial_archive

    solution_set = archive
    archive = update_archive(population, archive, portfolio, k, phi, ta, R)
    solution_set = update_ss(population, solution_set, portfolio, phi, tss, R)

    for i in range(generations):
        offspring_pop = offspring(population, archive, portfolio, k, phi, mutation_rate, R)

        # Evaluar población original y offspring
        combined = np.concatenate((population, offspring_pop), axis=0)
        fitness = evaluate(combined, archive, portfolio, k, phi, R).flatten()

        # Dividir fitness en mitades
        fitness_parents = fitness[:len(population)]
        fitness_offspring = fitness[len(population):]

        # Selección entre padre e hijo por posición
        new_population = []
        for j in range(len(population)):
            if fitness_offspring[j] > fitness_parents[j]:
                new_population.append(offspring_pop[j])
            else:
                new_population.append(population[j])
                
        population = np.array(new_population)

        archive = update_archive(population, archive, portfolio, k, phi, ta, R)
        solution_set = update_ss(population, solution_set, portfolio, phi, tss, R)


        # print(i)
        
        if (i+1) in [250, 500, 750, 1000]:
            name = portfolio[0].py_func.__name__
            np.save(f"{name}_{i+1}_gens_NS2_0_4.npy", solution_set)
            print(f"[Checkpoint] Guardado en generación {i+1}")

    return solution_set

SAVE THE FILES AND PLOT THE RESULTS

Save files

Plot PCA

In [ ]:
# @title
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import numpy as np

def plot_pca_groups(*data_matrices, labels=None):
    # Calcular los tamaños de cada grupo
    sizes = [len(dm) for dm in data_matrices]
    cumulative_sizes = np.cumsum([0] + sizes)

    # Combinar todas las matrices en una
    matrix = np.vstack(data_matrices)
    matrix = matrix.reshape(matrix.shape[0], -1)  # Asegura que es 2D

    # Estandarizar
    scaler = StandardScaler()
    matrix = scaler.fit_transform(matrix)

    # Aplicar PCA
    pca = PCA(n_components=2)
    reduced_data = pca.fit_transform(matrix)

    # Imprimir varianza explicada
    print("Varianza explicada acumulada:", np.cumsum(pca.explained_variance_ratio_))

    # Colores y marcadores predeterminados
    default_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b']
    default_markers = ['o', '*', 's', 'P', '^', 'x']

    # Graficar cada grupo
    plt.figure(figsize=(8, 5))
    for i in range(len(data_matrices)):
        group = reduced_data[cumulative_sizes[i]:cumulative_sizes[i+1], :]
        label = labels[i] if labels and i < len(labels) else f"Grupo {i+1}"
        color = default_colors[i % len(default_colors)]
        marker = default_markers[i % len(default_markers)]
        plt.scatter(group[:, 0], group[:, 1], s=1, marker=marker, label=label, color=color)

    plt.xlabel('x0', fontsize=12)
    plt.ylabel('x1', fontsize=12)
    plt.title(r'$\mathit{NS_{ls}}$ instances over feature space', fontsize=14)
    plt.legend(title="target", loc='best', frameon=True)
    plt.grid(False)
    plt.tight_layout()
    plt.show()


Compute descriptors

In [ ]:
# @title
def compute_descriptors(solution_set):

    N = solution_set.shape[1]
    mask = ~np.eye(N, dtype=bool)
    masked_data = solution_set[:,mask]
    flat = masked_data.reshape(masked_data.shape[0], -1)
    tens = torch.tensor(np.array(flat), dtype=torch.float32)
    ae_model.eval()
    with torch.no_grad():
        data_matrix = ae_model.encoder(tens).cpu().numpy().astype(np.float32)
    return data_matrix


In [ ]:
# import time
import numpy as np
from sklearn.model_selection import train_test_split

set_global_seed(42)

D = [50, 1]
N = 20
k = 3
phi = 0
generations = 1000
ta = 0.3
ts = 0.3
R=3

entrenamiento, populacion = train_test_split(train,test_size=D[0] / len(train),random_state=42)

print(entrenamiento.shape)
print(populacion.shape)

portfolio_1 = np.array([insert_ls, two_opt_ls, swap_ls])
portfolio_2 = np.array([swap_ls, two_opt_ls, insert_ls])
portfolio_3 = np.array([two_opt_ls, swap_ls, insert_ls])

solution_set_1 = novelty_search(D, N, k, phi, generations, portfolio_1, ta, ts, R, 1 / ((N * N) - N), [], populacion)
solution_set_2 = novelty_search(D, N, k, phi, generations, portfolio_2, ta, ts, R, 1 / ((N * N) - N), [], populacion)
solution_set_3 = novelty_search(D, N, k, phi, generations, portfolio_3, ta, ts, R, 1 / ((N * N) - N), [], populacion)

size_1 = len(solution_set_1)
size_2 = len(solution_set_2)
size_3 = len(solution_set_3)
print(f"Tamaños: {size_1}, {size_2}, {size_3}")

In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt
# from sklearn.model_selection import train_test_split

# # Asegurar comportamiento determinista
# set_global_seed(42)
# torch.use_deterministic_algorithms(True)
# os.environ["OMP_NUM_THREADS"] = "1"
# os.environ["MKL_NUM_THREADS"] = "1"
# torch.set_num_threads(1)

# # Parámetros fijos
# N = 20
# k = 3
# phi = 0.85
# generations = 40
# ta = 0.01
# ts = 0.03
# mutation_rate = 1 / ((N * N) - N)
# portfolios = [
#     np.array([insert_ls, two_opt_ls, swap_ls]),
#     np.array([swap_ls, two_opt_ls, insert_ls]),
#     np.array([two_opt_ls, swap_ls, insert_ls]),
# ]

# # Valores de D[0]
# D_values = list(range(2, 105, 6))
# mean_sizes_1 = []
# mean_sizes_2 = []
# mean_sizes_3 = []

# repetitions = 3

# for D0 in D_values:
#     print(f"\nEjecutando para D[0]={D0}")
#     D = [D0, 1]

#     sizes_1, sizes_2, sizes_3 = [], [], []

#     for r in range(repetitions):
#         set_global_seed(42 + r)  # Diferente semilla en cada repetición

#         entrenamiento, populacion = train_test_split(train, test_size=D[0] / len(train), random_state=42 + r)

#         # Asegurarse que la población sea par
#         if len(populacion) % 2 != 0:
#             populacion = populacion[:-1]

#         s1 = novelty_search(D, N, k, phi, generations, portfolios[0], ta, ts, mutation_rate, entrenamiento, populacion)
#         s2 = novelty_search(D, N, k, phi, generations, portfolios[1], ta, ts, mutation_rate, entrenamiento, populacion)
#         s3 = novelty_search(D, N, k, phi, generations, portfolios[2], ta, ts, mutation_rate, entrenamiento, populacion)

#         sizes_1.append(len(s1) - len(entrenamiento))
#         sizes_2.append(len(s2) - len(entrenamiento))
#         sizes_3.append(len(s3) - len(entrenamiento))

#     mean_sizes_1.append(np.mean(sizes_1))
#     mean_sizes_2.append(np.mean(sizes_2))
#     mean_sizes_3.append(np.mean(sizes_3))

# # Graficar los resultados
# plt.figure(figsize=(10, 6))
# plt.plot(D_values, mean_sizes_1, marker='o', label='Portfolio 1')
# plt.plot(D_values, mean_sizes_2, marker='s', label='Portfolio 2')
# plt.plot(D_values, mean_sizes_3, marker='^', label='Portfolio 3')
# plt.xlabel("Cantidad de Instancias Iniciales (D[0])")
# plt.ylabel("Promedio de Nuevas Instancias Generadas")
# plt.title("Promedio de instancias generadas vs. número de instancias iniciales")
# plt.legend()
# plt.grid(True)
# plt.tight_layout()
# plt.show()



In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt
# import time
# from sklearn.model_selection import train_test_split

# # Asegurar comportamiento determinista
# set_global_seed(42)
# torch.use_deterministic_algorithms(True)
# os.environ["OMP_NUM_THREADS"] = "1"
# os.environ["MKL_NUM_THREADS"] = "1"
# torch.set_num_threads(1)

# # Parámetros fijos
# N = 20
# k = 3
# phi = 0.85
# generations = 40
# ta = 0.01
# ts = 0.03
# mutation_rate = 1 / ((N * N) - N)
# portfolios = [
#     np.array([insert_ls, two_opt_ls, swap_ls]),
#     np.array([swap_ls, two_opt_ls, insert_ls]),
#     np.array([two_opt_ls, swap_ls, insert_ls]),
# ]

# # Valores de D[0]
# D_values = list(range(2, 105, 6))
# mean_times_1 = []
# mean_times_2 = []
# mean_times_3 = []

# repetitions = 3

# for D0 in D_values:
#     print(f"\nMidiendo tiempos para D[0]={D0}")
#     D = [D0, 1]

#     times_1, times_2, times_3 = [], [], []

#     for r in range(repetitions):
#         set_global_seed(42 + r)
#         entrenamiento, populacion = train_test_split(train, test_size=D[0] / len(train), random_state=42 + r)

#         # Asegurar población par
#         if len(populacion) % 2 != 0:
#             populacion = populacion[:-1]

#         # Portfolio 1
#         start = time.time()
#         _ = novelty_search(D, N, k, phi, generations, portfolios[0], ta, ts, mutation_rate, entrenamiento, populacion)
#         times_1.append(time.time() - start)

#         # Portfolio 2
#         start = time.time()
#         _ = novelty_search(D, N, k, phi, generations, portfolios[1], ta, ts, mutation_rate, entrenamiento, populacion)
#         times_2.append(time.time() - start)

#         # Portfolio 3
#         start = time.time()
#         _ = novelty_search(D, N, k, phi, generations, portfolios[2], ta, ts, mutation_rate, entrenamiento, populacion)
#         times_3.append(time.time() - start)

#     mean_times_1.append(np.mean(times_1))
#     mean_times_2.append(np.mean(times_2))
#     mean_times_3.append(np.mean(times_3))

# # Graficar los resultados
# plt.figure(figsize=(10, 6))
# plt.plot(D_values, mean_times_1, marker='o', label='Portfolio 1')
# plt.plot(D_values, mean_times_2, marker='s', label='Portfolio 2')
# plt.plot(D_values, mean_times_3, marker='^', label='Portfolio 3')
# plt.xlabel("Cantidad de Instancias Iniciales (D[0])")
# plt.ylabel("Tiempo promedio de ejecución (segundos)")
# plt.title("Tiempo de ejecución vs. número de instancias iniciales")
# plt.legend()
# plt.grid(True)
# plt.tight_layout()
# plt.show()


In [ ]:
data_matrix_0 = compute_descriptors(instancias)
data_aleatorio = compute_descriptors(aleatorio)
data_matrix_1 = compute_descriptors(solution_set_1)
data_matrix_2 = compute_descriptors(solution_set_2)
data_matrix_3 = compute_descriptors(solution_set_3)

In [ ]:
# 55# # Guardar archivos .npy con nombres adecuados
# np.save("900_two_opt_0_85", solution_set_3)
# np.save("900_swap_0_85", solution_set_2)
# np.save("900_insert_0_85", solution_set_1)

# # np.save(get_filename_descriptors(D, N, k, phi, generations, portfolio_1[0], ta, ts), data_matrix_1)
# # np.save(get_filename_descriptors(D, N, k, phi, generations, portfolio_2[0], ta, ts), data_matrix_2)
# # np.save(get_filename_descriptors(D, N, k, phi, generations, portfolio_3[0], ta, ts), data_matrix_3)

In [ ]:
plot_pca_groups(data_matrix_0, data_matrix_1, data_matrix_2, data_matrix_3, compute_descriptors(populacion))
plot_pca_groups(data_matrix_1, data_matrix_2)

In [ ]:
plot_pca_groups(compute_descriptors(entrenamiento))

In [ ]:
# Revisar por que a veces el novelty score es 0.

In [ ]:
# valor_insert = 0
# valor_swap = 0
# valor_two_opt = 0
# vector_clasificacion = np.zeros((len(validacion),1))
# for j in range(len(validacion)):
#   instance = validacion[j]
#   for i in range(10):
#     valor_insert = valor_insert + profit(instance, insert)
#     valor_swap = valor_swap + profit(instance, swap)
#     valor_two_opt = valor_two_opt + profit(instance, two_opt)
#   valor_insert = valor_insert / 10
#   valor_swap = valor_swap / 10
#   valor_two_opt = valor_two_opt / 10
#   if valor_insert > valor_swap and valor_insert > valor_two_opt:
#     vector_clasificacion[j] = 1
#   elif valor_swap > valor_insert and valor_swap > valor_two_opt:
#     vector_clasificacion[j] = 2
#   elif valor_two_opt > valor_insert and valor_two_opt > valor_swap:
#     vector_clasificacion[j] = 0
#   valor_insert = 0
#   valor_swap = 0
#   valor_two_opt = 0


In [ ]:
# import numpy as np
# from scipy.spatial.distance import cdist

# def clasificacion(populacion_desc, validacion):
#     # Paso 1: Calcular descriptores
#     descriptores_validacion = compute_descriptors(validacion)

#     # Paso 2: Calcular distancias (filas = validacion, columnas = poblacion)
#     distancias = cdist(descriptores_validacion, populacion_desc, metric='euclidean')

#     # Paso 3: Obtener índice del descriptor más cercano de la población
#     indices_minimos = np.argmin(distancias, axis=1)  # uno por cada fila de validación

#     return indices_minimos

In [ ]:
# prediccion = clasificacion(poblacion_desc, validacion)
# descriptores_validacion = compute_descriptors(validacion)
# print(len(prediccion))

In [ ]:
# prediccion_2 = np.zeros((len(prediccion),1))
# for i in range(len(prediccion)):
#   if prediccion[i] <= len_two_opt:
#     prediccion_2[i] = 0
#   elif prediccion[i] <= len_two_opt + len_insert and prediccion[i] > len_two_opt:
#     prediccion_2[i] = 1
#   else:
#     prediccion_2[i] = 2

# import numpy as np

# # Comparar elemento a elemento y contar coincidencias
# coinciden = np.sum(vector_clasificacion == prediccion_2)
# print("Coincidencias:", coinciden)


In [ ]:
# import numpy as np
# from scipy.spatial.distance import cdist

# def evaluar_clasificacion(validacion):
#     # Paso 1: Clasificación real según heurísticas (con profit promedio)
#     vector_clasificacion = np.zeros((len(validacion), 1))

#     for j in range(len(validacion)):
#         instance = validacion[j]
#         valor_insert = np.mean([profit(instance, insert) for _ in range(10)])
#         valor_swap = np.mean([profit(instance, swap) for _ in range(10)])
#         valor_two_opt = np.mean([profit(instance, two_opt) for _ in range(10)])

#         if valor_insert > valor_swap and valor_insert > valor_two_opt:
#             vector_clasificacion[j] = 1
#         elif valor_swap > valor_insert and valor_swap > valor_two_opt:
#             vector_clasificacion[j] = 2
#         else:
#             vector_clasificacion[j] = 0

#     distancias = cdist(descriptores_validacion, poblacion_desc, metric='euclidean')
#     indices_minimos = np.argmin(distancias, axis=1)

#     prediccion_2 = np.zeros((len(indices_minimos), 1))
#     for i in range(len(indices_minimos)):
#         if indices_minimos[i] < len_two_opt:
#             prediccion_2[i] = 0
#         elif indices_minimos[i] < len_two_opt + len_insert and indices_minimos[i] >= len_two_opt:
#             prediccion_2[i] = 1
#         else:
#             prediccion_2[i] = 2

#     # Paso 3: Comparar clasificaciones
#     coincidencias = np.sum(vector_clasificacion == prediccion_2)

#     print("Coincidencias:", coincidencias)
#     return coincidencias, vector_clasificacion, prediccion_2


In [ ]:
# coincidencias, _, _ = evaluar_clasificacion(validacion)
# print(coincidencias)

In [ ]:
# import numpy as np
# from sklearn.neighbors import KNeighborsClassifier

# def evaluar_clasificacion_knn(validacion, vector_clasificacion, k=3):
#     # Crear etiquetas de clase para la población
#     etiquetas_poblacion = np.concatenate((
#         np.zeros(len_two_opt),              # Clase 0: two_opt
#         np.ones(len_insert),                # Clase 1: insert
#         np.full(len_swap, 2)                # Clase 2: swap
#     ))

#     # Entrenar clasificador k-NN
#     clf = KNeighborsClassifier(n_neighbors=k, metric='euclidean')
#     clf.fit(poblacion_desc, etiquetas_poblacion)

#     # Predecir clases para las instancias de validación
#     predicciones = clf.predict(descriptores_validacion).reshape(-1, 1)

#     # Calcular coincidencias
#     coincidencias = np.sum(vector_clasificacion == predicciones)
#     print(f"Coincidencias con k={k}:", coincidencias)

#     return coincidencias, vector_clasificacion, predicciones


In [ ]:
# coincidencias_2, _, _ = evaluar_clasificacion_knn(validacion, vector_clasificacion, k=10000)

In [ ]:
# from google.colab import files
# files.download('AE_30_1_20_3_0.85_300_two_opt_0.01_0.03_descriptor.npy')  # por ejemplo: 'resultados.csv'

In [ ]:
# set_global_seed(42)
# set_global_seed_2(42)

# plot_autoencoder_latents(data_matrix_1, data_matrix_2, data_matrix_3)
# # plot_autoencoder_latents(solution_set_1, solution_set_2, solution_set_3)

GRAFCAR INSTANCIAS CONSEGUIDAS CON EL NS CON LOS DESCRIPTORES

In [ ]:
# from google.colab import files
# uploaded = files.upload()

In [ ]:
# import pickle

# # Replace 'novelty_search_LOP_4_alg.pkl' with the actual file name
# with open('NS_AE_20x20_seed_42.pkl', 'rb') as f:
#     data_1 = pickle.load(f)

# with open('novelty_search_LOP_4_alg.pkl', 'rb') as g:
#     data_2 = pickle.load(g)

# # Now you can use the 'data' object
# print(type(data_2))


In [ ]:
# import numpy as np

# # Guardar el diccionario como archivo .npy
# np.save('archivo_guardado_1.npy', data_1)
# np.save('archivo_guardado_2.npy', data_2)
# data_3 = np.load('instancias_aleatorias.npy')

# print(data_3.shape)

In [ ]:
# instancias_descriptores_1 = np.load('archivo_guardado_1.npy', allow_pickle=True)
# combined_array_1 = np.concatenate(
#     [data_1['2-opt'], data_1['swap'], data_1['insert']],
#     axis=0
# )
# instancias_descriptores_2 = np.load('archivo_guardado_2.npy', allow_pickle=True)
# combined_array_2 = np.concatenate(
#     [data_2['2-opt'], data_2['swap'], data_2['insert']],
#     axis=0
# )
# data_11 = data_1['2-opt']
# data_12 = data_1['swap']
# data_13 = data_1['insert']
# data_21 = data_2['2-opt']
# data_22 = data_2['swap']
# data_23 = data_2['insert']
# size_11 = len(data_11)
# size_12 = len(data_12)
# size_13 = len(data_13)
# size_21 = len(data_21)
# size_22 = len(data_22)
# size_23 = len(data_23)
# print(data_11.shape)
# print(data_12.shape)
# print(data_13.shape)
# print(data_21.shape)
# print(data_22.shape)
# print(data_23.shape)
# print(combined_array_1.shape)
# print(combined_array_2.shape)
# set_global_seed(42)
# set_global_seed_2(42)
# data_matrix_11 = compute_descriptors(data_11)
# data_matrix_12 = compute_descriptors(data_12)
# data_matrix_13 = compute_descriptors(data_13)
# data_matrix_21 = compute_descriptors(data_21)
# data_matrix_22 = compute_descriptors(data_22)
# data_matrix_23 = compute_descriptors(data_23)
# data_matrix_3 = compute_descriptors(data_3)



In [ ]:
# from sklearn.decomposition import PCA
# from sklearn.preprocessing import StandardScaler
# import matplotlib.pyplot as plt
# import numpy as np

# matrix_11 = data_matrix_11
# matrix_12 = data_matrix_12
# matrix_13 = data_matrix_13
# matrix_21 = data_matrix_21
# matrix_22 = data_matrix_22
# matrix_23 = data_matrix_23

# matrix = np.vstack((matrix_11, matrix_12, matrix_13, matrix_21, matrix_22, matrix_23, data_matrix_3))

# set_global_seed(42)
# set_global_seed_2(42)

# scaler = StandardScaler()
# matrix = scaler.fit_transform(matrix)

# pca = PCA()
# reduced_data = pca.fit_transform(matrix)

# group_11 = reduced_data[0:size_11, :]
# group_12 = reduced_data[size_11:size_11+size_12, :]
# group_13 = reduced_data[size_11+size_12:size_11+size_12+size_13, :]
# group_21 = reduced_data[size_11+size_12+size_13:size_11+size_12+size_13+size_21, :]
# group_22 = reduced_data[size_11+size_12+size_13+size_21:size_11+size_12+size_13+size_21+size_22, :]
# group_23 = reduced_data[size_11+size_12+size_13+size_21+size_22:size_11+size_12+size_13+size_21+size_22+size_23, :]
# group_3 = reduced_data[size_11+size_12+size_13+size_21+size_22+size_23:,:]

# print("Varianza explicada acumulada:", np.cumsum(pca.explained_variance_ratio_))

# plt.figure(figsize=(8, 5))

# # plt.scatter(group_11[:, 0], group_11[:, 1], s=20, marker='o', label="two_opt_AE", color='#1f77b4')   # Azul
# # plt.scatter(group_12[:, 0], group_12[:, 1], s=20, marker='o', label="swap_AE", color='#ff7f0e')       # Naranja
# # plt.scatter(group_13[:, 0], group_13[:, 1], s=20, marker='o', label="insert_AE", color='#2ca02c')       # Verde
# plt.scatter(group_21[:, 0], group_21[:, 1], s=20, marker='+', label="two_opt", color='#1f77b4')   # Azul
# plt.scatter(group_22[:, 0], group_22[:, 1], s=20, marker='+', label="swap", color='#ff7f0e')       # Naranja
# plt.scatter(group_23[:, 0], group_23[:, 1], s=20, marker='+', label="insert", color='#2ca02c')       # Verde

# # Limitar ejes:
# plt.xlim(-6, 4)   # Cambia los valores según tu rango deseado en el eje x
# plt.ylim(-5, 4)   # Cambia los valores según tu rango deseado en el eje y

# plt.xlabel('x0', fontsize=12)
# plt.ylabel('x1', fontsize=12)
# plt.title(r'$\mathit{NS_{ls}}$ instances over feature space', fontsize=14)
# plt.legend(loc='best', frameon=True)
# plt.grid(False)
# plt.tight_layout()
# plt.show()

In [ ]:
# plt.scatter(group_11[:, 0], group_11[:, 1], s=20, marker='o', label="two_opt_AE", color='#1f77b4')   # Azul
# plt.scatter(group_12[:, 0], group_12[:, 1], s=20, marker='o', label="swap_AE", color='#ff7f0e')       # Naranja
# plt.scatter(group_13[:, 0], group_13[:, 1], s=20, marker='o', label="insert_AE", color='#2ca02c')       # Verde
# # plt.scatter(group_21[:, 0], group_21[:, 1], s=20, marker='+', label="two_opt", color='#1f77b4')   # Azul
# # plt.scatter(group_22[:, 0], group_22[:, 1], s=20, marker='+', label="swap", color='#ff7f0e')       # Naranja
# # plt.scatter(group_23[:, 0], group_23[:, 1], s=20, marker='+', label="insert", color='#2ca02c')       # Verde

# # Limitar ejes:
# plt.xlim(-6, 4)   # Cambia los valores según tu rango deseado en el eje x
# plt.ylim(-5, 4)   # Cambia los valores según tu rango deseado en el eje y

# plt.xlabel('x0', fontsize=12)
# plt.ylabel('x1', fontsize=12)
# plt.title(r'$\mathit{NS_{ls}}$ instances over feature space', fontsize=14)
# plt.legend(loc='best', frameon=True)
# plt.grid(False)
# plt.tight_layout()
# plt.show()

In [ ]:
# plt.scatter(group_11[:, 0], group_11[:, 1], s=20, marker='o', label="two_opt_AE", color='#1f77b4')   # Azul
# plt.scatter(group_12[:, 0], group_12[:, 1], s=20, marker='o', label="swap_AE", color='#ff7f0e')       # Naranja
# plt.scatter(group_13[:, 0], group_13[:, 1], s=20, marker='o', label="insert_AE", color='#2ca02c')       # Verde
# plt.scatter(group_21[:, 0], group_21[:, 1], s=20, marker='+', label="two_opt", color='#1f77b4')   # Azul
# plt.scatter(group_22[:, 0], group_22[:, 1], s=20, marker='+', label="swap", color='#ff7f0e')       # Naranja
# plt.scatter(group_23[:, 0], group_23[:, 1], s=20, marker='+', label="insert", color='#2ca02c')       # Verde

# # Limitar ejes:
# plt.xlim(-6, 4)   # Cambia los valores según tu rango deseado en el eje x
# plt.ylim(-5, 4)   # Cambia los valores según tu rango deseado en el eje y

# plt.xlabel('x0', fontsize=12)
# plt.ylabel('x1', fontsize=12)
# plt.title(r'$\mathit{NS_{ls}}$ instances over feature space', fontsize=14)
# plt.legend(loc='best', frameon=True)
# plt.grid(False)
# plt.tight_layout()
# plt.show()

In [ ]:
# # plt.scatter(group_11[:, 0], group_11[:, 1], s=20, marker='o', label="two_opt_AE", color='#1f77b4')   # Azul
# # plt.scatter(group_12[:, 0], group_12[:, 1], s=20, marker='o', label="swap_AE", color='#ff7f0e')       # Naranja
# # plt.scatter(group_13[:, 0], group_13[:, 1], s=20, marker='o', label="insert_AE", color='#2ca02c')       # Verde
# # plt.scatter(group_21[:, 0], group_21[:, 1], s=20, marker='+', label="two_opt", color='#1f77b4')   # Azul
# # plt.scatter(group_22[:, 0], group_22[:, 1], s=20, marker='+', label="swap", color='#ff7f0e')       # Naranja
# # plt.scatter(group_23[:, 0], group_23[:, 1], s=20, marker='+', label="insert", color='#2ca02c')       # Verde
# plt.scatter(group_3[:, 0], group_3[:, 1], s=20, marker='+', label="LOLIB", color='#2ca02c')       # Verde

# plt.xlabel('x0', fontsize=12)
# plt.ylabel('x1', fontsize=12)
# plt.title(r'$\mathit{NS_{ls}}$ instances over feature space', fontsize=14)
# plt.legend(loc='best', frameon=True)
# plt.grid(False)
# plt.tight_layout()
# plt.show()

In [ ]:
# plt.scatter(group_11[:, 0], group_11[:, 1], s=20, marker='o', label="two_opt_AE", color='#1f77b4')   # Azul
# plt.scatter(group_12[:, 0], group_12[:, 1], s=20, marker='o', label="swap_AE", color='#ff7f0e')       # Naranja
# plt.scatter(group_13[:, 0], group_13[:, 1], s=20, marker='o', label="insert_AE", color='#2ca02c')       # Verde
# plt.scatter(group_21[:, 0], group_21[:, 1], s=20, marker='+', label="two_opt", color='#1f77b4')   # Azul
# plt.scatter(group_22[:, 0], group_22[:, 1], s=20, marker='+', label="swap", color='#ff7f0e')       # Naranja
# plt.scatter(group_23[:, 0], group_23[:, 1], s=20, marker='+', label="insert", color='#2ca02c')       # Verde
# plt.scatter(group_3[:, 0], group_3[:, 1], s=20, marker='+', label="LOLIB", color='#2ca02c')       # Verde

# plt.xlabel('x0', fontsize=12)
# plt.ylabel('x1', fontsize=12)
# plt.title(r'$\mathit{NS_{ls}}$ instances over feature space', fontsize=14)
# plt.legend(loc='best', frameon=True)
# plt.grid(False)
# plt.tight_layout()
# plt.show()

In [ ]:
# from sklearn.cluster import KMeans

# n_clusters = 3
# kmeans = KMeans(n_clusters=n_clusters, random_state=42)
# labels = kmeans.fit_predict(matrix)

# num_zeros = (labels == 1).sum()
# print(num_zeros)


In [ ]:
# !pip install --upgrade --force-reinstall scikit-learn-extra

In [ ]:
# from sklearn_extra.cluster import KMedoids
# from sklearn.metrics import pairwise_distances

# kmedoids = KMedoids(n_clusters=3, metric='manhattan', random_state=42)
# labels = kmedoids.fit_predict(matrix)

# num_zeros = (labels == 0).sum()
# print(num_zeros)



In [ ]:
# import matplotlib.pyplot as plt

# # reduced_matrix: datos reducidos a 2D usando PCA
# # labels: resultado de kmeans.fit_predict(...)

# plt.figure(figsize=(8, 6))

# Puntos coloreados por su cluster
# plt.scatter(reduced_data[:, 0], reduced_data[:, 1],
#             c=labels, cmap='viridis', s=50)

# plt.title('Clusters visualizados en 2D con PCA')
# plt.xlabel('Componente Principal 1')
# plt.ylabel('Componente Principal 2')
# plt.grid(True)
# plt.show()


In [ ]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F

# set_global_seed(42)
# set_global_seed_2(42)

# class Autoencoder(nn.Module):
#     def __init__(self, input_dim, latent_dim, hidden_dim=50):
#         super(Autoencoder, self).__init__()

#         self.encoder = nn.Sequential(
#             nn.Linear(input_dim, hidden_dim),
#             nn.ReLU(),
#             nn.Linear(hidden_dim, latent_dim)
#         )

#         self.decoder = nn.Sequential(
#             nn.Linear(latent_dim, hidden_dim),
#             nn.ReLU(),
#             nn.Linear(hidden_dim, input_dim),
#             nn.Tanh()
#         )

#     def forward(self, x):
#         z = self.encoder(x)
#         out = self.decoder(z)
#         return out

# def ae_loss(recon_x, x):
#     return F.mse_loss(recon_x, x, reduction='sum')

# def train_autoencoder(model, data, epochs=50, batch_size=64, lr=1e-3):
#     optimizer = torch.optim.Adam(model.parameters(), lr=lr)
#     for epoch in range(epochs):
#         model.train()
#         total_loss = 0
#         for i in range(0, len(data), batch_size):
#             batch = data[i:i+batch_size]
#             optimizer.zero_grad()
#             recon = model(batch)
#             loss = ae_loss(recon, batch)
#             loss.backward()
#             optimizer.step()
#             total_loss += loss.item()
#         # print(f'Epoch {epoch+1}, Loss: {total_loss / len(data):.4f}')

# def get_ae_latents(model, data, batch_size=64):
#     model.eval()
#     latents = []
#     with torch.no_grad():
#         for i in range(0, len(data), batch_size):
#             batch = data[i:i+batch_size]
#             z = model.encoder(batch)
#             latents.append(z)
#     return torch.cat(latents, dim=0)

# matrix = np.vstack((data_1, data_2, data_3))
# flat_data = matrix.reshape(matrix.shape[0], -1)
# data = torch.tensor(flat_data, dtype=torch.float32)


# input_dim = data.shape[1]
# latent_dim = 2


# ae_model_graph = Autoencoder(input_dim=input_dim, latent_dim=latent_dim)


# train_autoencoder(ae_model_graph, data, epochs=50, batch_size=64)


# z = get_ae_latents(ae_model_graph, data, 64)
# print(z.shape)  # (num_instancias, latent_dim)

# group_1 = z[0:size_11, :]
# group_2 = z[size_11:size_11+size_22, :]
# group_3 = z[size_11+size_22:, :]
# # group_4 = z[size_11+size_22+size_33:, :]

In [ ]:
# plt.figure(figsize=(8, 6))
# plt.scatter(group_1[:, 0], group_1[:, 1], label="Grupo 1", alpha=0.7)
# plt.scatter(group_2[:, 0], group_2[:, 1], label="Grupo 2", alpha=0.7)
# plt.scatter(group_3[:, 0], group_3[:, 1], label="Grupo 3", alpha=0.7)
# # plt.scatter(group_4[:, 0], group_4[:, 1], label="Grupo 4", alpha=0.7)

# plt.title("Espacio Latente (Autoencoder)")
# plt.xlabel("Latente 1")
# plt.ylabel("Latente 2")
# plt.legend()
# plt.grid(True)
# plt.tight_layout()
# plt.show()

In [ ]:
# z_np = z.detach().cpu().numpy()
# labels_np = labels.numpy() if isinstance(labels, torch.Tensor) else np.array(labels)

In [ ]:
# import matplotlib.pyplot as plt

# plt.figure(figsize=(8, 6))

# scatter = plt.scatter(z_np[:, 0], z_np[:, 1], c=labels_np, cmap='viridis', s=50)
# plt.colorbar(scatter, label='Clusters / Labels')

# plt.title('Latentes del Autoencoder con Labels')
# plt.xlabel('Latente 1')
# plt.ylabel('Latente 2')
# plt.grid(True)
# plt.show()


En vez de usar K-means (No supervisado), voy a intentar hacer un modelo de clasificacion supervisada, usando el rendom forest

Distribucion empirica

In [ ]:
# instancias_insert = np.load('instancias_insert.npy')
# instancias_swap = np.load('instancias_swap.npy')
# instancias_two_opt = np.load('instancias_two_opt.npy')
# len_insert = instancias_insert.shape[0]
# len_swap = instancias_swap.shape[0]
# len_two_opt = instancias_two_opt.shape[0]
# poblacion_desc = np.concatenate((instancias_two_opt, instancias_insert, instancias_swap), axis=0)
# print(len_insert)
# print(len_swap)
# print(len_two_opt)
# print(poblacion_desc.shape)

In [ ]:
# import numpy as np

# def valores_sin_diagonal(entrenamiento):
#     # entrenamiento: array 3D (n_matrices, N, N)
#     n_matrices, N, _ = entrenamiento.shape

#     # Creamos una máscara para valores no diagonales
#     mask = np.ones((N, N), dtype=bool)
#     np.fill_diagonal(mask, False)

#     # Extraemos valores no diagonales de cada matriz y aplanamos
#     valores = entrenamiento[:, mask].flatten()
#     return valores


In [ ]:
0.4988-0.4644